# Electoral College voting problem
[Riddler Express](https://https://fivethirtyeight.com/features/are-you-a-pinball-wizard/), July 24, 2020



>Riddler Township is having its quadrennial presidential election. Each of the town’s 10 “shires” is allotted a certain number of electoral votes: two, plus one additional vote for every 10 citizens (rounded to the nearest 10).  
>The names and populations of the 10 shires are summarized in the table below.
>Which way will Riddler Township swing?
!<img src="riddlertownship.png" width = "400"/>)
>As you may know, under this sort of electoral system, it is quite possible for a presidential candidate to lose the popular vote and still win the election.  
>If there are two candidates running for president of Riddler Township, and every single citizen votes for one or the other, then what is the lowest percentage of the popular vote that a candidate can get while still winning the election?

## How should we solve this?
We will use integer optimization to solve the LP. In essence, we seek to minimize the popular vote while ensuring an electoral college victory. Since Riddler Township has a total of 75 electoral votes we know in order to win our candidate must have at least 38 electoral votes. Conveniently, each shire is a "winner take all" electoral shire (the majority popular vote candidate gets all the electoral votes).

For a given shire we define the following variables: 

*   `lose_ub` is the upper bound on the number of popular votes guaranteeing a loss in a given shire.
*   `win_lb` is the lower bound on the number of popular votes guaranteeing a win in a given shire. 

We have an objective function and constraints emerging. 

## Formulating the Integer Linear Program
We have $N$ shires. Define:

The number of popular votes in Shire $i$ for our candidate.

$$
\begin{equation*}
\ v_{i} \text{ for $i$...$N$}\\
\end{equation*}
$$

For every Shire $i$:
$$
  \begin{equation}
    x_{i}=
    \begin{cases}
      1, & \text{if shire $i$ is a win}\\
      0, & \text{if shire $i$ is a loss}
    \end{cases}
  \end{equation}
$$

Our object is to minimize the popular votes:
$$
\begin{equation*}
\sum_{i=1}^{N} v_{i} 
\end{equation*}
$$

The first constraint is that the popular votes, in shire $i$ for our candidate cannot exceed the population of shire $i$(given in the table), and must be non-negative.  
 $$0 \leq v_{i} \leq pop_{i} \text{    for $i$...$N$}$$   


Our candidate must also win the electoral vote (get at least 38 ec votes). $e_{i}$ is the number of electoral votes for shire $i$ (given in the table).
$$
\begin{equation*}
\sum_{i=1}^{N} e_{i}x_{i} \geq 38
\end{equation*}
$$

Next we must add constraints which represent mapping of popular votes to electoral wins or losses:


*   (a.) **If our candidate wins the popular vote, then they get the electoral votes** or if $v_{i}$ $\geq$ `win_lb`, then $x_{1} = 1$


* (b.)  **If our candidate loses the popular vote, then they do not get the electoral votes** or if $v_{i}$ $\leq$ `lose_ub`, then $x_{1} = 0$

Both (a.) and (b.) are logic statements, and must be converted to constraints. (a.) is of the form if $a^Tx \geq b$ then $z$ = $1$, which becomes $a^Tx - b \leq Mz - (1-z)$ and (b.) is of the form if $a^Tx \leq b$ then $z$ = $0$, which becomes $a^Tx - b \geq m(1-z) + z$, where $M$ and $m$ are the upper and lower bound of $a^Tx-b$. 

With some simplification we get:  
(a.) the constraint representing a loss  
*   $ v_{i} \geq $ `win_lb` $x_{i}$ for $i$...$N$

(b.) the constraint representing a win  
*   $ v_{i} \leq $ `win_lb` $x_{i} + $ `loss_ub` for $i$...$N$













In [1]:
# Import PuLP modeler functions

from pulp import *

In [2]:
#Create the 'prob' variable to contain the problem data
prob = LpProblem("Riddler_Township_EC", LpMinimize)

In [3]:
pop = [11,21,31,41,51,61,71,81,91,101] #population for shire i
e = [3,4,5,6,7,8,9,10,11,12] #electoral votes for shire i
lose_ub = [(val-1)/2 for val in pop] #upper bound for losing popular vote for shire i
win_lb = [val+1 for val in lose_ub] #lower bound for winnig popular vote for shire i

In [4]:
#Create the variable Xi using list comp so we get x_1... x_10
x = LpVariable.dicts("x",[val for val in range (1,11)],0,1,LpInteger)
#Create the variable Xi using list comp so we get v_1... v_10
v = LpVariable.dicts("v",[val for val in range (1,11)],0,None,LpInteger)

In [5]:
#Add the objective function
prob += lpSum(v[i] for i in range(1,11))

In [6]:
# Add constraints

#constraint for v that the number of votes in each shire be capped at population of shire
for i in range(0,10):
  prob += v[i+1] <= pop[i]

#constraint that our candidate must win
prob += lpSum(x[i]*e[i-1] for i in range(1,11)) >= 38

#logical constraint for winning (a.)
for i in range(0,10):
  prob += v[i+1] <= win_lb[i]*x[i+1] +lose_ub[i]

#logical constraint for losing (b.)
for i in range(0,10):
  prob += v[i+1] >= win_lb[i]*x[i+1]




In [7]:
prob.solve()

1

In [8]:
#Print the variables
for v in prob.variables():
  print(v.name, "=", v.varValue)

v_1 = 6.0
v_10 = 0.0
v_2 = 11.0
v_3 = 16.0
v_4 = 21.0
v_5 = 0.0
v_6 = 0.0
v_7 = 36.0
v_8 = 0.0
v_9 = 46.0
x_1 = 1.0
x_10 = 0.0
x_2 = 1.0
x_3 = 1.0
x_4 = 1.0
x_5 = 0.0
x_6 = 0.0
x_7 = 1.0
x_8 = 0.0
x_9 = 1.0


We have a total of **136 popular votes** and **38 electoral votes** representing **24%** of the population.